In [ ]:
import numpy as np
import torch
import os
import random
from shutil import copyfile
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.models as models
from tqdm import tqdm
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision import datasets
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, f1_score, auc, precision_score, recall_score
import matplotlib.pyplot as plt
import torchvision.models as models
from PIL import Image
import cv2
from torchvision.utils import save_image
from true_classify import *
from Utils import *
from anonymization_methods import *
from datasets import *
from torchvision.transforms.functional import to_pil_image
from collections import Counter
import matplotlib.image as mpimg
import time
import matplotlib.pyplot as plt
import xlrd
import openpyxl
import pandas as pd
from PIL import ImageDraw
import torchshow as ts

In [ ]:
# define training and testing data directories


source_path = 'Path to the data'

class_names = [folder for folder in os.listdir(source_path) if os.path.isdir(os.path.join(source_path, folder))]
num_classes = len(class_names)

file_list = os.listdir(source_path)

# if needed to update the code
# model_dir = 'The name of the AI model'


# the path where the data will be saved
output_path = 'path'



# save_roc_dir = 'D:/Younas_Work/D2_Final/Results_Final/ROC/ROC Plots Mouth_Masked/'
# excel_file_path = 'D:/Younas_Work/D2_Final/Results_Final/Excel_Sheets/Mouth_Masked.xlsx'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
Define a new transform with additional data augmentations
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# model = models.convnext_base(pretrained=True)
# model.classifier[2]=nn.Linear(1024,num_classes)

# model.load_state_dict(torch.load('Convnext_pretrained_younas.pt'))
# model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.001)

In [ ]:
#Setting Parameter values

# we will put all the values to minimize the changes.

#For Simple Blur
blur_kernel_size = 3

#For DP Blur
#block
b = 16
#m & eps are for privacy budget. Larger m = more noise
m = 2
#eps is the privacy parameter in DP, smaller eps means higher noise and more distortion
eps = 100
#K is kernel size for gaussian blur. Height and width of the Gaussian kernel
k = 9
#SD of Gaussian blur. Larger sigma means more blurring
sigma = 0

#For Simple Pixelate
pixel_size = 15


#For DP Pixelate
#b, m and eps are the same for DP pixelate and DP Blur

In [ ]:
batch_size = 1
test_loader = create_test_loader(source_path, batch_size)
counter = 0
for data, target in local_test_loader:
    data = data.to(device)
    target = target.to(device)
    ############ choose the method you need
    perturbed = create_blurred_image(data, blur_kernel_size)
#     perturbed = create_pixelated_image(data, pixel_size)
#     perturbed = create_dp_blurred_image(data, b, m, eps, k, sigma)
#     perturbed = create_dp_pixelated_image(data, b, m, eps)
#     perturbed = draw_points_on_image(annonymized_image, optimized_gradients[0, 0, :, :] >= 1, point_size=0.05, point_color=(255, 0, 0))
#     perturbed =mask_mouth(x, mouth_cascade, ds_factor=0.75)
#     perturbed =mask_eyes(x, eye_cascade, radius, color, thickness)
    
    class_label = target.to(cpu)
    class_subfolder = class_names[class_label]            

    class_output_path = os.path.join(output_path, class_subfolder)
    if not os.path.exists(class_output_path):
        os.makedirs(class_output_path)
    final_path = os.path.join(class_output_path, str(counter))
    ts.save(perturbed,final_path+".jpg")
    counter +=1